## Run large section of brain through model to identify subvolumes that include cells: return location of cells in the brain

In [1]:
import sys
sys.path.append("/Users/johnduva/Desktop/Git/NeuroData/brainlit") 

from brainlit.utils.session import NeuroglancerSession
# from brainlit.utils.swc import graph_to_paths
import napari
import pandas as pd
import numpy as np
import os
import PIL
from PIL import Image
import matplotlib.pyplot as plt
import random
import cv2
from cloudvolume import CloudVolume, view
from tifffile import imsave
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import sklearn.metrics

/Users/johnduva/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


## Generate subvolumes for testing

In [3]:
dir = "s3://open-neurodata/brainlit/brain1"
vol = CloudVolume(dir, parallel=False, mip=2, progress=True, fill_missing=True)

x1 = 3700; x2 = 4700;
y1 = 2700; y2 = 3700;
z1 = 1164; z2 = 2164;
image = vol[x1:x2, y1:y2, z1:z2] 

Downloading: 3194it [00:40, 78.34it/s]                          


In [6]:
# (x2-x1)*(y2-y1)*(z2-z1)
final = []
finalIDX = []
for x in range(0, x2-x1-12, 12 ):
    for y in range(0, y2-y1-12, 12 ):
        for z in range(0, z2-z1-12, 12 ):
            test = image[x:x+12, y:y+12, z:z+12] 
#             if np.all(np.asarray(test) == 0):
#                 print([x,y,z])
#             else:  

            # Create array of subvolumes to test
            final.append(test)
            finalIDX.append([x1+x, y1+y, z1+z])

#             fig = plt.figure()
#             f, axarr = plt.subplots(1,3, figsize=(12,12) )  
#             axarr[0].set_title("x=%d:%d, y=%d:%d, z=%d" % (x1+x,x1+x+12, y1+y,y1+y+12, z1+z+3) ) 
#             axarr[1].set_title("x=%d:%d, y=%d:%d, z=%d" % (x1+x,x1+x+12, y1+y,y1+y+12, z1+z+6) ) 
#             axarr[2].set_title("x=%d:%d, y=%d:%d, z=%d" % (x1+x,x1+x+12, y1+y,y1+y+12, z1+z+9) ) 
#             axarr[0].imshow(test[:,:,3] )
#             axarr[1].imshow(test[:,:,6] )
#             axarr[2].imshow(test[:,:,9] )

 # *Test model on current dataset*

In [89]:
# Convert to desired shape: n x 12 x 12 x 12 x 1
final2 = np.zeros((len(final), 12, 12,12,1))
for subv in final:
    final2[i] = np.asarray(subv)
final2 = np.squeeze(final2)

(571777, 12, 12, 12)

In [90]:
# ROI_TEST = ROI_TEST.astype(np.float32)
print("Number of Items in Test Data: %s" %len(final2))
final2 = np.expand_dims(final2, axis=1)
print('Shape of Test Data Vector:', final2.shape)

Number of Items in Test Data: 571777
Shape of Test Data Vector: (571777, 1, 12, 12, 12)


## Build Neural Net Class

In [91]:
# m1 = torch.load('model2.pt')
# 1. Model Architecture
# INPUT: 
# - 1 x 12 x 12 x 12 image
# CONV1: 3d CONV
# MAXPOOL: 3d MP
# CONV2: 3d CONV
# CONV3: 3d CONV
# FC1: Fully Connected Layer
# FC2: Fully Connected Layer

class M1(nn.Module):
    def __init__(self):
        super(M1, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv3d(1,64, (5,5,5),padding = 2),nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.pool = nn.MaxPool3d((2, 2, 2),2) # kernel, stride
        self.conv2 = nn.Sequential(nn.Conv3d(64, 64, (3,3,3), padding = 1), nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.conv3 = nn.Sequential(nn.Conv3d(64, 64, (3,3,3), padding = 1), nn.Dropout(dr)) # in_channel, out_channel, kernel
        self.fc1 = nn.Sequential(nn.Linear(6*6*6*64,150),nn.Dropout(dr))
        self.fc2 = nn.Sequential(nn.Linear(150,1),nn.Dropout(dr))

    def forward(self,x):
#        print('Input Shape: ', x.shape)
        x = F.relu(self.conv1(x))
#        print('Shape after CONV1: ', x.shape)
        # Conv1 Activation: ReLU
        x = self.pool(x)
#         print('Shape after Maxpool: ', x.shape)
        # Followed by Maxpool
     
        x = F.relu(self.conv2(x))
#         print('Shape after CONV2: ', x.shape)
        # Conv2 Activation: ReLU
        
        x = F.relu(self.conv3(x))
#         print('Shape after CONV3: ', x.shape)
        # Conv3 Activation: ReLU
        
        x = x.reshape(x.size(0),-1)
#         print('Shape after flatten: ', x.shape)
        # flatten vector
              
        x = F.relu(self.fc1(x))
        # FC1 Activation: ReLU
#         print('Shape after FC1: ', x.shape)
        x = self.fc2(x)
        # FC2 Activation: None
#         print('Shape after FC2: ', x.shape)
        m = nn.Sigmoid()
#         print('Shape after Sigmoid Output: ', x.shape)

        x = m(x)
        # Output Activation: Sigmoid
        return x

#Hyperparamters
num_epochs = 200
batch_size = 5
learning_rate = 1e-3
momentum = 0.9
dr = 0.3
weight_decay = 1e-4
m1 = M1()

## Load pre-trained model

In [96]:
m1 = M1()
m1.load_state_dict(torch.load('crossvalidation4.pt', map_location = torch.device('cpu') ))
m1 = m1.double()

## Reformat data for pyTorch and run it through m1 model

In [144]:
final2 = final2.astype(np.float32)
labels=[]
for i in range(100000): # for i in range(len(final2)):
    w = torch.from_numpy(final2[i])
    w = w.unsqueeze(1)
    w = w.double()
    print(w.shape)
    print(' ')
    w = m1(w)
    labels.append(w.detach().numpy())

labels = np.asarray(labels)
labels = labels.squeeze()

torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size

torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size

torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size([1, 1, 12, 12, 12])
 
torch.Size

KeyboardInterrupt: 

In [143]:
isinstance(final2[0], np.float64)

False

## Compute False/Positive Rate and True/Positive Rate using Receiver Operating Characteristics and Precision Recall pairs

In [67]:
# fpr, tpr, thresholds = sklearn.metrics.roc_curve(truelabels, labels)
mets = sklearn.metrics.precision_recall_curve(truelabels, labels, pos_label=None, sample_weight=None)
#np.save('positivecells.npy',positivecells)

# *Results*

## Asssess accuracy with Area Under Curve score

In [68]:
auc = sklearn.metrics.roc_auc_score(truelabels, labels)
print('AUC: %.3f' % auc)

AUC: 0.859


# Plot False/Positive Rate vs. True/Positive Rate

In [19]:
# plt.plot(fpr, tpr, marker='.')
# plt.title("AUC")
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# # show the plot
# plt.show()

# Calculate and plot F1 score using Recall vs. Precision

In [20]:
# precision, recall, thresholds = sklearn.metrics.precision_recall_curve(truelabels, labels)
# auc = sklearn.metrics.auc(recall, precision)

In [21]:
# plt.plot(recall, precision, marker='.', label='Logistic')
# # axis labels
# plt.xlabel('Recall')
# plt.ylabel('Precision')
# plt.show()

# *Conclusion*

### Our results show a consistently high degree of accuracy when utilizing the Area Under Curve score as such a measure: between 85% and 90%. This confirms our hypothesis that this model generalizes well to other datasets and implies that it has the potential to be a useful methodological addition to the Brainlit package. Some possible improvements would be: (1) increase the model's input volume size from 12x12x12 (by retraining with larger subvolumes), and (2) consider retraining with additional data from our brains. 